In [4]:
# backtest_dma2green_single_run.py
"""
Single-run backtester for DMA(10,20) + strict 2-green strategy with Exit v2.
- Entry: 10DMA crosses above 20DMA AND two consecutive green candles where 2nd candle's O/H/L/C > 1st
- Exit v2:
    * Two consecutive red candles AND second closes below 20DMA, OR
    * (optional) protective exit: any close below 20DMA
- Optional time exit: max hold N days (default 10)
- Optional HSL/TSL (hard/trailing stop)
- Optional indicator filters (RSI/MACD/ADX/SMA/Bollinger) combined via 'all' or 'any'
- Groww-like fees + GST + STT + DP + (optional) slippage; per-trade isolated capital sizing
- Warm cache (parquet preferred, CSV fallback)
"""

import os
import time
import pathlib
import pandas as pd
import numpy as np
import yfinance as yf

# -------------------- CONFIG --------------------
TICKER_LIST = ['BSE.NS', 'BAJFINANCE.NS', 'BAJAJFINSV.NS', 'BDL.NS', 'BEL.NS', 'BHARTIARTL.NS', 'CHOLAFIN.NS',
               'COFORGE.NS', 'DIVISLAB.NS', 'DIXON.NS', 'NYKAA.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'ICICIBANK.NS',
               'INDHOTEL.NS', 'INDIGO.NS', 'KOTAKBANK.NS', 'MFSL.NS', 'MAXHEALTH.NS', 'MAZDOCK.NS', 'MUTHOOTFIN.NS',
               'PAYTM.NS', 'PERSISTENT.NS', 'SBICARD.NS', 'SBILIFE.NS', 'SRF.NS', 'SHREECEM.NS', 'SOLARINDS.NS',
               'TVSMOTOR.NS', 'UNITDSPR.NS']

START_DATE = "2025-01-01"
END_DATE   = "2025-08-01"

# Strategy params
DMA_FAST = 10
DMA_SLOW = 20

# Capital per trade (isolated sizing)
CAPITAL_PER_TRADE = 50000.0

# -------------------- Toggles --------------------
# Exits
USE_PROTECTIVE_EXIT = False      # exit if any Close < 20DMA
USE_TIME_EXIT       = True      # force-close after MAX_HOLD_DAYS
MAX_HOLD_DAYS       = 10

USE_HARD_STOP       = True     # hard stop as % from entry
HARD_STOP_PCT       = 5.0       # only used if USE_HARD_STOP

USE_TRAILING_STOP   = False     # trailing stop as % from peak after entry
TRAILING_STOP_PCT   = 10.0      # only used if USE_TRAILING_STOP

# Indicator filters (entry-time)
INDICATORS = {
    "rsi":  {"enabled": True,  "length": 14, "min": 45},     # require RSI >= min
    "macd": {"enabled": False, "fast": 12, "slow": 26, "signal": 9},  # MACD>signal & hist>0
    "adx":  {"enabled": False,  "length": 14, "min_adx": 20}, # require ADX>=min & +DI > -DI
    "sma":  {"enabled": True, "length": 50},                 # require Close >= SMA_len
    "bb":   {"enabled": False, "length": 20, "stddev": 2.0, "rule": "above_mid"}  # 'above_mid'/'below_mid'
}
COMBINATION_MODE = "all"  # 'all' or 'any'

# Fees & slippage toggles
APPLY_FEES      = True
APPLY_SLIPPAGE  = True

# -------------------- FEES & SLIPPAGE (Groww-like) --------------------
SLIPPAGE_PCT = 0.05  # percent per side (adverse)

# Brokerage: "₹20 OR 0.1% per executed order — whichever is lower, minimum ₹5"
BROKERAGE_CAP_RUPEES = 20.0
BROKERAGE_PCT_CAP    = 0.1   # percent
BROKERAGE_MIN_RUPEES = 5.0

# Regulatory / exchange (approx values; per side)
EXCHANGE_TXN_PCT   = 0.00297  # percent
SEBI_TURNOVER_PCT  = 0.0001   # percent
IPFT_PCT           = 0.0001   # percent

# STT for delivery sell
STT_SELL_PCT       = 0.025    # percent on sell notional

# Stamp duty (buy side)
STAMP_DUTY_BUY_PCT = 0.003    # percent

# DP charge (sell, delivery)
DP_CHARGE_SELL     = 16.5     # rupees per sell if notional >= 100

# GST on (brokerage + exch + sebi + ipft + dp)
GST_PCT            = 18.0     # percent

# -------------------- CACHE HELPERS --------------------
CACHE_DIR = "cache"
pathlib.Path(CACHE_DIR).mkdir(parents=True, exist_ok=True)

_parquet_engine = None
try:
    import pyarrow  # noqa: F401
    _parquet_engine = "pyarrow"
except Exception:
    try:
        import fastparquet  # noqa: F401
        _parquet_engine = "fastparquet"
    except Exception:
        _parquet_engine = None

if _parquet_engine:
    print(f"Cache: using parquet engine '{_parquet_engine}'.")
else:
    print("Cache: parquet engine not found — falling back to CSV cache (no extra deps required).")

def ensure_cache_dir():
    pathlib.Path(CACHE_DIR).mkdir(parents=True, exist_ok=True)

def _cache_path_for_ticker(ticker, use_parquet):
    safe = ticker.replace('/', '_').replace(':', '_')
    if use_parquet:
        return os.path.join(CACHE_DIR, f"{safe}.parquet")
    else:
        return os.path.join(CACHE_DIR, f"{safe}.csv")

def download_with_retry(ticker, start, end, interval="1d", max_retries=3, backoff_sec=2):
    attempt = 0
    while attempt < max_retries:
        try:
            df = yf.download(ticker, start=start, interval=interval, end=end,
                             auto_adjust=True, progress=False, threads=True, multi_level_index=False)
            return df
        except Exception as e:
            attempt += 1
            wait = backoff_sec * attempt
            print(f"  download error for {ticker} (attempt {attempt}/{max_retries}): {e}. retrying in {wait}s")
            time.sleep(wait)
    print(f"  download failed for {ticker} after {max_retries} attempts.")
    return None

def _read_cache(ticker):
    use_parquet = _parquet_engine is not None
    path = _cache_path_for_ticker(ticker, use_parquet)
    alt = _cache_path_for_ticker(ticker, not use_parquet)

    paths_to_try = []
    if os.path.exists(path):
        paths_to_try.append(path)
    if os.path.exists(alt):
        paths_to_try.append(alt)

    for p in paths_to_try:
        try:
            if os.path.getsize(p) == 0:
                print(f"  Warning: cache file {p} is zero bytes — removing.")
                try:
                    os.remove(p)
                except Exception as e:
                    print(f"    Could not remove zero-byte file {p}: {e}")
                continue
        except OSError:
            continue

        try:
            if p.endswith('.parquet'):
                return pd.read_parquet(p, engine=_parquet_engine)
            else:
                return pd.read_csv(p, index_col=0, parse_dates=True)
        except Exception as e:
            print(f"  Warning: failed to read cache for {ticker} at {p}: {e}")
            try:
                os.remove(p)
                print(f"    Removed corrupted cache file {p}.")
            except Exception as rm_e:
                print(f"    Could not remove corrupted cache file {p}: {rm_e}")
            continue
    return None

def _write_cache(ticker, df):
    use_parquet = _parquet_engine is not None
    path = _cache_path_for_ticker(ticker, use_parquet)
    tmp_path = path + ".tmp"
    try:
        os.makedirs(os.path.dirname(path), exist_ok=True)
        if use_parquet:
            df.to_parquet(tmp_path, engine=_parquet_engine)
        else:
            df.to_csv(tmp_path)
        os.replace(tmp_path, path)
        return True
    except Exception as e:
        print(f"  Warning: failed to write cache for {ticker} to {path}: {e}")
        try:
            if os.path.exists(tmp_path):
                os.remove(tmp_path)
        except Exception:
            pass
        if use_parquet:
            try:
                alt = _cache_path_for_ticker(ticker, use_parquet=False)
                tmp_alt = alt + ".tmp"
                df.to_csv(tmp_alt)
                os.replace(tmp_alt, alt)
                print(f"  Wrote CSV fallback cache for {ticker} to {alt}")
                return True
            except Exception as e2:
                print(f"  Warning: fallback CSV write also failed for {ticker}: {e2}")
        return False

def warm_cache(tickers, start=START_DATE, end=END_DATE):
    print(f"Warm cache: downloading {len(tickers)} tickers sequentially ...")
    ensure_cache_dir()
    for t in tickers:
        try:
            existing = _read_cache(t)
            if existing is not None:
                continue
            df = download_with_retry(t, start, end, interval="1d", max_retries=3, backoff_sec=2)
            if df is None or df.empty:
                print(f"  warm_cache: no data for {t}; skipping.")
                continue
            df = normalize_df_columns(df)
            df = collapse_duplicate_columns_take_first(df)
            ok = _write_cache(t, df)
            if not ok:
                print(f"  warm_cache: failed to write cache for {t}")
            else:
                print(f"  warm_cache: cached {t}")
            time.sleep(0.25)
        except Exception as e:
            print(f"  warm_cache: error for {t}: {e}")
            continue
    print("Warm cache: done.")

# -------------------- NORMALIZATION & DUPLICATE HANDLING --------------------
def normalize_df_columns(df):
    if hasattr(df, "columns") and getattr(df.columns, "nlevels", 1) > 1:
        df.columns = ["_".join([str(c) for c in col if c is not None]).strip() for col in df.columns.values]

    cols = list(df.columns)
    mapping = {}
    lower_map = {c.lower(): c for c in cols}
    for name in ['close', 'high', 'low', 'open', 'volume']:
        if name in lower_map:
            mapping[lower_map[name]] = name.capitalize()
        else:
            match = next((c for c in cols if name in c.lower()), None)
            if match:
                mapping[match] = name.capitalize()
    if mapping:
        df = df.rename(columns=mapping)
    return df

def collapse_duplicate_columns_take_first(df):
    if df.columns.duplicated().any():
        dup_names = list({c for c in df.columns[df.columns.duplicated()]})
        print(f"  Warning: duplicate columns found and collapsed for: {dup_names}")
        df = df.groupby(df.columns, axis=1).first()
    return df

# -------------------- INDICATORS --------------------
def sma(series, length):
    return series.rolling(length).mean()

def rsi(df, length=14, column='Close'):
    series = df[column]
    delta = series.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.ewm(alpha=1/length, adjust=False).mean()
    avg_loss = loss.ewm(alpha=1/length, adjust=False).mean()
    rs = avg_gain / (avg_loss.replace(0, np.nan))
    df['RSI'] = (100 - (100 / (1 + rs))).fillna(50)
    return df

def macd(df, fast=12, slow=26, signal=9, column='Close'):
    ema_fast = df[column].ewm(span=fast, adjust=False).mean()
    ema_slow = df[column].ewm(span=slow, adjust=False).mean()
    macd_line = ema_fast - ema_slow
    signal_line = macd_line.ewm(span=signal, adjust=False).mean()
    df['MACD'] = macd_line
    df['MACD_signal'] = signal_line
    df['MACD_hist'] = df['MACD'] - df['MACD_signal']
    return df

def adx(df, n=14):
    high = df['High']; low = df['Low']; close = df['Close']
    tr1 = high - low
    tr2 = (high - close.shift(1)).abs()
    tr3 = (low - close.shift(1)).abs()
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    up_move = high.diff(); down_move = -low.diff()
    plus_dm = np.where((up_move > down_move) & (up_move > 0), up_move, 0.0)
    minus_dm = np.where((down_move > up_move) & (down_move > 0), down_move, 0.0)
    tr_smooth = pd.Series(tr).rolling(window=n).sum()
    plus_dm_smooth = pd.Series(plus_dm).rolling(window=n).sum()
    minus_dm_smooth = pd.Series(minus_dm).rolling(window=n).sum()
    plus_di = 100 * (plus_dm_smooth / tr_smooth).replace([np.inf, -np.inf], 0).fillna(0)
    minus_di = 100 * (minus_dm_smooth / tr_smooth).replace([np.inf, -np.inf], 0).fillna(0)
    dx = (abs(plus_di - minus_di) / (plus_di + minus_di)).replace([np.inf, -np.inf], 0) * 100
    df['+DI'] = plus_di; df['-DI'] = minus_di; df['ADX'] = dx.rolling(window=n).mean()
    return df

def bbands(df, length=20, stddev=2.0, column='Close'):
    mid = df[column].rolling(length).mean()
    std = df[column].rolling(length).std()
    df['BB_middle'] = mid
    df['BB_upper'] = mid + stddev*std
    df['BB_lower'] = mid - stddev*std
    return df

# -------------------- STRATEGY BUILDING BLOCKS --------------------
def two_green_strict(df):
    # True at t if t-1 and t are green, and candle t has O/H/L/C > candle t-1 O/H/L/C
    o, h, l, c = df['Open'], df['High'], df['Low'], df['Close']
    green_prev = df['Close'].shift(1) > df['Open'].shift(1)
    green_now  = df['Close'] > df['Open']
    cond_strict = (o > o.shift(1)) & (h > h.shift(1)) & (l > l.shift(1)) & (c > c.shift(1))
    return (green_prev & green_now & cond_strict)

def two_red(df):
    red_prev = df['Close'].shift(1) < df['Open'].shift(1)
    red_now  = df['Close'] < df['Open']
    return (red_prev & red_now)

def build_base_signals(df, dma_fast=DMA_FAST, dma_slow=DMA_SLOW):
    df['DMA_fast'] = sma(df['Close'], dma_fast)
    df['DMA_slow'] = sma(df['Close'], dma_slow)
    df['dma_cross_up'] = (df['DMA_fast'] > df['DMA_slow']) & (df['DMA_fast'].shift(1) <= df['DMA_slow'].shift(1))
    df['two_green_strict'] = two_green_strict(df)
    df['two_red'] = two_red(df)
    return df

# -------------------- DATA PREP (cache-aware) --------------------
def get_stock_data(ticker, start, end):
    ensure_cache_dir()
    raw_df = _read_cache(ticker)
    if raw_df is None:
        raw_df = download_with_retry(ticker, start, end, interval="1d", max_retries=3, backoff_sec=2)
        if raw_df is None or raw_df.empty:
            return None
        raw_df = normalize_df_columns(raw_df)
        raw_df = collapse_duplicate_columns_take_first(raw_df)
        ok = _write_cache(ticker, raw_df)
        if not ok:
            print(f"  Warning: failed to cache {ticker} (continuing without cache).")

    df = raw_df.copy()
    df = normalize_df_columns(df)
    df = collapse_duplicate_columns_take_first(df)

    for c in ['Close', 'High', 'Low', 'Open', 'Volume']:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors='coerce')

    df = build_base_signals(df, dma_fast=DMA_FAST, dma_slow=DMA_SLOW)

    # Optional indicators
    if INDICATORS.get('rsi', {}).get('enabled', False):
        df = rsi(df, length=INDICATORS['rsi'].get('length', 14))

    if INDICATORS.get('macd', {}).get('enabled', False):
        df = macd(df,
                  fast=INDICATORS['macd'].get('fast', 12),
                  slow=INDICATORS['macd'].get('slow', 26),
                  signal=INDICATORS['macd'].get('signal', 9))

    if INDICATORS.get('adx', {}).get('enabled', False):
        df = adx(df, n=INDICATORS['adx'].get('length', 14))

    if INDICATORS.get('sma', {}).get('enabled', False):
        df[f"SMA_{INDICATORS['sma'].get('length', 50)}"] = df['Close'].rolling(INDICATORS['sma'].get('length', 50)).mean()

    if INDICATORS.get('bb', {}).get('enabled', False):
        df = bbands(df, length=INDICATORS['bb'].get('length', 20), stddev=INDICATORS['bb'].get('stddev', 2.0))

    df.dropna(inplace=True)
    if df.empty:
        return None
    return df

# -------------------- FEES & SLIPPAGE HELPERS --------------------
def groww_brokerage_for_order(order_value):
    pct_based = (BROKERAGE_PCT_CAP / 100.0) * order_value
    capped = min(BROKERAGE_CAP_RUPEES, pct_based)
    return max(capped, BROKERAGE_MIN_RUPEES)

def compute_pnl(entry_price, exit_price, qty, apply_fees=APPLY_FEES, apply_slippage=APPLY_SLIPPAGE):
    # slippage per side
    entry_eff = entry_price * (1 + (SLIPPAGE_PCT/100.0 if apply_slippage else 0.0))
    exit_eff  = exit_price  * (1 - (SLIPPAGE_PCT/100.0 if apply_slippage else 0.0))

    entry_notional = entry_eff * qty
    exit_notional  = exit_eff  * qty

    gross_profit = exit_notional - entry_notional

    if not apply_fees:
        return entry_eff, exit_eff, gross_profit, 0.0, {"fees_total": 0.0}

    brok_buy  = groww_brokerage_for_order(entry_notional)
    brok_sell = groww_brokerage_for_order(exit_notional)

    exch_buy  = (EXCHANGE_TXN_PCT  / 100.0) * entry_notional
    exch_sell = (EXCHANGE_TXN_PCT  / 100.0) * exit_notional
    sebi_buy  = (SEBI_TURNOVER_PCT / 100.0) * entry_notional
    sebi_sell = (SEBI_TURNOVER_PCT / 100.0) * exit_notional
    ipft_buy  = (IPFT_PCT / 100.0) * entry_notional
    ipft_sell = (IPFT_PCT / 100.0) * exit_notional
    stamp_buy = (STAMP_DUTY_BUY_PCT / 100.0) * entry_notional
    stt_sell  = (STT_SELL_PCT / 100.0) * exit_notional
    dp_sell   = DP_CHARGE_SELL if exit_notional >= 100.0 else 0.0

    taxable = brok_buy + brok_sell + exch_buy + exch_sell + sebi_buy + sebi_sell + ipft_buy + ipft_sell + dp_sell
    gst = (GST_PCT / 100.0) * taxable

    total_fees = brok_buy + brok_sell + exch_buy + exch_sell + sebi_buy + sebi_sell + ipft_buy + ipft_sell + stamp_buy + stt_sell + dp_sell + gst
    net_profit = gross_profit - total_fees

    details = {
        "brokerage_buy": brok_buy, "brokerage_sell": brok_sell,
        "exchange_buy": exch_buy, "exchange_sell": exch_sell,
        "sebi_buy": sebi_buy, "sebi_sell": sebi_sell,
        "ipft_buy": ipft_buy, "ipft_sell": ipft_sell,
        "stamp_buy": stamp_buy, "stt_sell": stt_sell,
        "dp_sell": dp_sell, "gst": gst, "fees_total": total_fees
    }
    return entry_eff, exit_eff, net_profit, total_fees, details

# -------------------- ENTRY FILTERS --------------------
def indicator_entry_checks(row, prev_row):
    checks = {}

    if INDICATORS.get('rsi', {}).get('enabled', False):
        rsi_min = INDICATORS['rsi'].get('min', 45)
        r = row.get('RSI', np.nan)
        checks['rsi'] = (r >= rsi_min)

    if INDICATORS.get('macd', {}).get('enabled', False):
        checks['macd'] = (row.get('MACD', 0) > row.get('MACD_signal', 0)) and (row.get('MACD_hist', 0) > 0)

    if INDICATORS.get('adx', {}).get('enabled', False):
        adx_min = INDICATORS['adx'].get('min_adx', 20)
        checks['adx'] = (row.get('ADX', 0) >= adx_min) and (row.get('+DI', 0) > row.get('-DI', 0))

    if INDICATORS.get('sma', {}).get('enabled', False):
        sma_len = INDICATORS['sma'].get('length', 50)
        checks['sma'] = (row['Close'] >= row.get(f"SMA_{sma_len}", np.nan))

    if INDICATORS.get('bb', {}).get('enabled', False):
        rule = INDICATORS['bb'].get('rule', 'above_mid')
        if rule == 'above_mid':
            checks['bb'] = (row['Close'] >= row.get('BB_middle', np.nan))
        else:
            checks['bb'] = (row['Close'] <= row.get('BB_middle', np.nan))

    return checks

def combine_indicator_checks(checks, mode='all'):
    if not checks:
        return True
    vals = list(checks.values())
    return all(vals) if mode == 'all' else any(vals)

# -------------------- BACKTEST --------------------
def run_backtest_on_df(df, ticker, capital_per_trade=CAPITAL_PER_TRADE):
    in_pos = False
    trades = []
    entry_price = 0.0; entry_date = None; qty = 0
    peak_price = 0.0; trailing_stop_price = 0.0; hard_stop_price = 0.0
    days_in_trade = 0

    tsl_mult = 1 - (TRAILING_STOP_PCT / 100.0) if USE_TRAILING_STOP else None
    hsl_mult = 1 - (HARD_STOP_PCT / 100.0) if USE_HARD_STOP else None

    for i in range(1, len(df)):
        prev_row = df.iloc[i-1]
        row = df.iloc[i]

        # ENTRY
        base_entry = bool(row['dma_cross_up'] and row['two_green_strict'])

        ind_checks = indicator_entry_checks(row, prev_row)
        enabled_any = any(v.get('enabled', False) for v in INDICATORS.values())
        ind_ok = combine_indicator_checks(ind_checks, mode=COMBINATION_MODE) if enabled_any else True

        entry_signal = base_entry and ind_ok

        if (not in_pos) and entry_signal:
            entry_price = row['Close']
            expected_entry_eff = entry_price * (1 + (SLIPPAGE_PCT/100.0 if APPLY_SLIPPAGE else 0.0))
            qty = int(capital_per_trade // expected_entry_eff)
            if qty <= 0:
                continue
            in_pos = True
            entry_date = row.name
            days_in_trade = 0

            if USE_TRAILING_STOP:
                peak_price = entry_price
                trailing_stop_price = peak_price * tsl_mult
            if USE_HARD_STOP:
                hard_stop_price = entry_price * hsl_mult

        elif in_pos:
            days_in_trade += 1

            # Update TSL
            if USE_TRAILING_STOP and row['High'] > peak_price:
                peak_price = row['High']
                trailing_stop_price = peak_price * tsl_mult

            # EXIT CONDITIONS
            exit_price = 0.0
            exit_reason = None

            # 1) Hard stop
            if USE_HARD_STOP and row['Low'] <= hard_stop_price:
                exit_price = hard_stop_price; exit_reason = "HSL"
            # 2) Trailing stop
            elif USE_TRAILING_STOP and row['Low'] <= trailing_stop_price:
                exit_price = trailing_stop_price; exit_reason = "TSL"
            # 3) Time exit
            elif USE_TIME_EXIT and days_in_trade >= MAX_HOLD_DAYS:
                exit_price = row['Close']; exit_reason = "Time"
            else:
                # 4) Exit v2 (two red and second close < 20DMA)
                exit_two_red_below_20 = bool(row['two_red'] and (row['Close'] < row['DMA_slow']))
                # 5) Protective exit (any close < 20DMA)
                exit_protective = bool(row['Close'] < row['DMA_slow']) if USE_PROTECTIVE_EXIT else False

                if exit_two_red_below_20:
                    exit_price = row['Close']; exit_reason = "TwoRedBelow20"
                elif exit_protective:
                    exit_price = row['Close']; exit_reason = "Protective"

            # Execute exit
            if exit_price > 0:
                entry_eff, exit_eff, pnl_currency, fees_total, fee_break = compute_pnl(entry_price, exit_price, qty,
                                                                                      apply_fees=APPLY_FEES,
                                                                                      apply_slippage=APPLY_SLIPPAGE)
                roi_pct_on_capital = (pnl_currency / capital_per_trade) * 100.0

                trades.append({
                    "ticker": ticker,
                    "entry_date": entry_date, "exit_date": row.name,
                    "qty": qty,
                    "entry_price": entry_price, "exit_price": exit_price,
                    "entry_eff": entry_eff, "exit_eff": exit_eff,
                    "net_profit": pnl_currency, "fees_total": fees_total,
                    "roi_%": roi_pct_on_capital,
                    "exit_reason": exit_reason,
                    "days_held": days_in_trade
                })

                # reset
                in_pos = False
                entry_price = 0.0; entry_date = None; qty = 0
                peak_price = 0.0; trailing_stop_price = 0.0; hard_stop_price = 0.0
                days_in_trade = 0

    return trades

# -------------------- SUMMARY --------------------
def aggregate_metrics(trades):
    if not trades:
        return {
            "total_trades": 0, "win_rate": 0.0, "avg_roi_pct": 0.0,
            "total_net_profit": 0.0, "overall_roi_pct": 0.0,
            "avg_holding_days": 0.0, "time_exits": 0, "two_red_exits": 0, "protective_exits": 0,
            "hsl_exits": 0, "tsl_exits": 0
        }
    df = pd.DataFrame(trades)
    total_trades = len(df)
    win_rate = (df['net_profit'] > 0).mean() * 100.0
    avg_roi = df['roi_%'].mean()
    total_net = df['net_profit'].sum()
    overall_roi = (total_net / (CAPITAL_PER_TRADE * total_trades)) * 100.0 if total_trades else 0.0
    avg_hold = df['days_held'].mean()

    return {
        "total_trades": int(total_trades),
        "win_rate": float(win_rate),
        "avg_roi_pct": float(avg_roi),
        "total_net_profit": float(total_net),
        "overall_roi_pct": float(overall_roi),
        "avg_holding_days": float(avg_hold),
        "time_exits": int((df['exit_reason'] == 'Time').sum()),
        "two_red_exits": int((df['exit_reason'] == 'TwoRedBelow20').sum()),
        "protective_exits": int((df['exit_reason'] == 'Protective').sum()),
        "hsl_exits": int((df['exit_reason'] == 'HSL').sum()),
        "tsl_exits": int((df['exit_reason'] == 'TSL').sum())
    }

# -------------------- MAIN --------------------
if __name__ == "__main__":
    print("Starting single-run backtest for DMA(10,20)+2-green with full toggles.")
    print(f"Dates: {START_DATE} to {END_DATE} | Capital/trade: ₹{CAPITAL_PER_TRADE:,.0f}")
    print(f"Exits: protective={USE_PROTECTIVE_EXIT}, time_exit={USE_TIME_EXIT} ({MAX_HOLD_DAYS}d), HSL={USE_HARD_STOP}, TSL={USE_TRAILING_STOP}")
    print(f"Filters enabled: {[k for k,v in INDICATORS.items() if v.get('enabled', False)]} | mode={COMBINATION_MODE}")
    print(f"Fees={'ON' if APPLY_FEES else 'OFF'}, Slippage={'ON' if APPLY_SLIPPAGE else 'OFF'}")

    # Warm cache (optional but recommended once)
    warm_cache(TICKER_LIST)

    all_trades = []
    for t in TICKER_LIST:
        try:
            df = get_stock_data(t, START_DATE, END_DATE)
        except Exception as e:
            print(f"[{t}] data error: {e}")
            df = None
        if df is None:
            print(f"[{t}] skipped (no data).")
            continue
        trades = run_backtest_on_df(df, t, capital_per_trade=CAPITAL_PER_TRADE)
        print(f"[{t}] trades: {len(trades)}")
        all_trades.extend(trades)

    # Summary
    print("\n=== Portfolio Summary ===")
    M = aggregate_metrics(all_trades)
    for k,v in M.items():
        print(f"{k}: {v}")

    # Save
    if all_trades:
        out = pd.DataFrame(all_trades)
        out.sort_values(['ticker','entry_date'], inplace=True)
        out.to_csv("dma2green_single_results.csv", index=False)
        print("\nSaved trade log to dma2green_single_results.csv")
    else:
        print("\nNo trades produced with current settings.")


Cache: using parquet engine 'pyarrow'.
Starting single-run backtest for DMA(10,20)+2-green with full toggles.
Dates: 2025-01-01 to 2025-08-01 | Capital/trade: ₹50,000
Exits: protective=False, time_exit=True (10d), HSL=True, TSL=False
Filters enabled: ['rsi', 'sma'] | mode=all
Fees=ON, Slippage=ON
Warm cache: downloading 30 tickers sequentially ...
Warm cache: done.
[BSE.NS] trades: 9
[BAJFINANCE.NS] trades: 10
[BAJAJFINSV.NS] trades: 12
[BDL.NS] trades: 10
[BEL.NS] trades: 13
[BHARTIARTL.NS] trades: 10
[CHOLAFIN.NS] trades: 19
[COFORGE.NS] trades: 11
[DIVISLAB.NS] trades: 16
[DIXON.NS] trades: 7
[NYKAA.NS] trades: 4
[HDFCBANK.NS] trades: 14
[HDFCLIFE.NS] trades: 7
[ICICIBANK.NS] trades: 20
[INDHOTEL.NS] trades: 19
[INDIGO.NS] trades: 6
[KOTAKBANK.NS] trades: 8
[MFSL.NS] trades: 17
[MAXHEALTH.NS] trades: 6
[MAZDOCK.NS] trades: 4
[MUTHOOTFIN.NS] trades: 16
[PAYTM.NS] trades: 5
[PERSISTENT.NS] trades: 8
[SBICARD.NS] trades: 4
[SBILIFE.NS] trades: 9
[SRF.NS] trades: 8
[SHREECEM.NS] trades: